<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np
import io

### Load the Dataset and Visualise

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
# df_train.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/Train_20K.csv?token=As1EmHVd_04h8pxz2Rr5aHldCB3DdRV1ks5cn3UowA%3D%3D"
df_train = pd.read_csv(url, sep='\t')

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_val = pd.read_csv(io.StringIO(uploaded['check.csv'].decode('utf-8')), sep='\t')
# df_val.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/check.csv?token=As1EmGDbgpndOWYhmX4VqD7m4N3zVaa0ks5cn3UrwA%3D%3D"
df_val = pd.read_csv(url, sep='\t')

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
# df_test.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/Test_5K.csv?token=As1EmLgiSJtng6_pWA552sBVIdXROwEgks5cn3XOwA%3D%3D"
df_test = pd.read_csv(url, sep='\t')

In [49]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))
print('Number of movie reviews', len(df_train['label']))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005
Number of movie reviews 17999


### Data pre-processing

In [50]:
raw_text_reviews = df_train['text'].astype(str).tolist()
len(raw_text_reviews), raw_text_reviews[-2]

(17999,
 'Anyone notice that Tommy only has 3 facial expressions.<br /><br />1. The angry eyes look he gives every enemy. 2. The holding of the hands to face, mouth agape and frightened eyes. 3. The smiling Tommy Turnbull.<br /><br />I have to say that i pretty much hate this show, i don\'t watch it but it\'s like Code Lyoko, we\'ve all watched at least one, i must say that this show is borderline racist, uninteresting and pointless.<br /><br />every episode ends with robotboy winning, except for one exception when robotboy basically let this overly geeky freakazoid fly away on a jetpack.<br /><br />The jokes are pretty crude too, i think it\'s mostly people saying the word "Suck" or farting, i think the bullies of the show are quite shocking too.<br /><br />Isn\'t there one that hides a bowling ball under his hate, and the other uses a chain, for god sake, what kind of school is he going to. Not to mention his older brother, who is borderline psychopathic and has no other character qu

In [83]:
train_text_labels = df_train['label'].astype(int)
len(train_text_labels), train_text_labels[len(train_text_labels)-2]

(17999, 0)

In [0]:
import re
import string
def preprocess_document(doc):
  # negative sense should not be eleminated + some short representation
  CONTRACTIONS = {"mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they have",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'am":"i am", "it's":"it is"}
  punctuation = string.punctuation + "\n\n"
  punc_replace = ''.join([' ' for s in punctuation]); # required for replacing punctuation with null ('')
  doc_clean = doc.replace('-', ' '); # replace - with null str
  doc_clean = (doc_clean.encode('ascii', 'ignore')).decode("utf-8")
  doc_clean = doc_clean.replace('<br />', ''); # replace <br /> with ''
  doc_clean = doc_clean.replace("’", "'"); # replace <br /> with null str
  doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
  doc_clean = " ".join(doc_clean) # list to sentence of strings
  doc_clean = re.sub(r'\W +', ' ', doc_clean) # except [a-zA-Z0-9_]
  doc_clean = re.sub(r'\d+', ' ', doc_clean) # remove numbers [0-9]
  trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
  doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')]);
  doc_clean = doc_clean.split(' ');
  doc_clean = [word for word in doc_clean if len(word) > 0];

  return doc_clean;

In [85]:
train_text_reviews = [preprocess_document(review.lower()) for review in raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (text_labels[len(text_labels)-2])

['anyone', 'notice', 'that', 'tommy', 'only', 'has', 'facial', 'expressions', 'the', 'angry', 'eyes', 'look', 'he', 'gives', 'every', 'enemy', 'the', 'holding', 'of', 'the', 'hands', 'to', 'face', 'mouth', 'agape', 'and', 'frightened', 'eyes', 'the', 'smiling', 'tommy', 'turnbull', 'i', 'have', 'to', 'say', 'that', 'i', 'pretty', 'much', 'hate', 'this', 'show', 'i', 'do', 'not', 'watch', 'it', 'but', 'it', 'is', 'like', 'code', 'lyoko', 'we', 'have', 'all', 'watched', 'at', 'least', 'one', 'i', 'must', 'say', 'that', 'this', 'show', 'is', 'borderline', 'racist', 'uninteresting', 'and', 'pointless', 'every', 'episode', 'ends', 'with', 'robotboy', 'winning', 'except', 'for', 'one', 'exception', 'when', 'robotboy', 'basically', 'let', 'this', 'overly', 'geeky', 'freakazoid', 'fly', 'away', 'on', 'a', 'jetpack', 'the', 'jokes', 'are', 'pretty', 'crude', 'too', 'i', 'think', 'it', 'is', 'mostly', 'people', 'saying', 'the', 'word', 'suck', 'or', 'farting', 'i', 'think', 'the', 'bullies', 'of

###Creating Bag Of Word (BOW) representation of sentences.

In [86]:
# Fetching the tokens (all words in a doc) and types(unique words in a doc) in the document
types = ()
tokens = []
for row in train_text_reviews:
    tokens += [words for words in row]
types = set(tokens)

VOCAB_SIZE, TOKEN_SIZE = len(types), len(tokens)
VOCAB_SIZE, TOKEN_SIZE

(65218, 4275850)

In [96]:
word_to_ix = {x:i for i, x in enumerate(types)}
ix_to_word = [x for i, x in enumerate(types)]

ix_to_word[word_to_ix['kick']]=='kick', word_to_ix['kick'], len(ix_to_word)

(True, 32089, 65218)

In [97]:
for i, (key, value) in enumerate(word_to_ix.items()):
  if(i > 10):
    break
  print (key,':', value)

vanesa : 0
ignacio : 1
specialty : 2
arkham : 3
bathhouses : 4
muccino : 5
aggravate : 6
nudity : 7
ssg : 8
hitchhikers : 9
alaska : 10


## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # for automatically implement backprop


##Model Definition for the BOWClassifier

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    SEED = 42
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    self.lin = nn.Linear(vocab_size, num_labels) # initialises weights and biases
    
  def forward(self, x):
    return F.softmax(self.lin(x), dim=1)

## Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            raise ValueError('Word',word,' not present in the dictionary. Sorry!')
        else:
            vec[word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])

In [164]:
NUM_LABELS = 2
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE) # initialises weights and biases
bow.lin # desc linear layer

Linear(in_features=65218, out_features=2, bias=True)

In [165]:
# visualise the parameter
for param in bow.parameters(): # desc the parameter value
    print(param,param.size())

Parameter containing:
tensor([[ 0.0030,  0.0033, -0.0009,  ..., -0.0018, -0.0014,  0.0025],
        [-0.0028, -0.0018, -0.0008,  ...,  0.0033,  0.0018,  0.0016]],
       requires_grad=True) torch.Size([2, 65218])
Parameter containing:
tensor([ 0.0021, -0.0028], requires_grad=True) torch.Size([2])


### Let's check with a sample case how our model is working

In [166]:
sample_data, sample_label = train_text_reviews[0], train_text_labels[0]
bow_vector = torch.autograd.Variable(make_bow_vector(sample_data, word_to_ix))
print (bow_vector)
logprobs = bow.forward(bow_vector) # calls forward function
print(train_text_reviews[0])
print(logprobs.data)

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
['john', 'waters', 'has', 'given', 'us', 'a', 'genuinely', 'enjoyable', 'film', 'this', 'certainly', 'is', 'not', 'without', 'its', 'shocking', 'waters', 'esque', 'moments', 'but', 'it', 'is', 'tamer', 'than', 'his', 'older', 'culty', 'stuff', 'such', 'as', 'pink', 'flamingoes', 'pecker', 'harkens', 'back', 'to', 'john', 's', 'early', 'mainstream', 'stage', 'in', 'that', 'it', 'reminds', 'the', 'viewer', 'of', 'the', 'same', 'kind', 'of', 'humor', 'that', 'was', 'evident', 'in', 'polyester', 'overall', 'a', 'really', 'fun', 'comedy', 'with', 'some', 'great', 'moments']
tensor([[0.5013, 0.4987]])


### Testing with normal variable

In [167]:
bow_vec = Variable(make_bow_vector(preprocess_document(df_train['text'][1].lower()), word_to_ix)) # encoding test data
logprobs = bow(bow_vec) # call to forward
print(logprobs)
pred = np.argmax(logprobs.data.numpy())
print('prediction: {}'.format(pred))
print('actual: {}'.format(df_train['label'][0]))


bow_vec = Variable(make_bow_vector(preprocess_document(df_train['text'][10].lower()), word_to_ix)) # encoding test data
logprobs = bow(bow_vec) # call to forward
print(logprobs)
pred = np.argmax(logprobs.data.numpy())
print('prediction: {}'.format(pred))
print('actual: {}'.format(df_train['label'][0]))

tensor([[0.5022, 0.4978]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 1
tensor([[0.4940, 0.5060]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1


### Define Loss fun

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

## Train the model

In [0]:
# the training loop
for epoch in range(1):
  print(epoch)
  for i, instance in enumerate(train_text_reviews):
    # get the training data
    label = train_text_labels[i]
#     print ('actual label :', label)
    bow.zero_grad()
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    label = Variable(make_target(label))
#     print ('tensor actual label :', label)
    probs = bow(bow_vec) # forward pass
    print ('pred :', probs)
    loss = loss_function(probs, label)
    loss.backward()
#     print('CURRENT LOSS: {}'.format(loss.data))
    opt.step()
